## TextRank

Устанавливаем и импортируем необходивые библиотеки.

In [49]:
# Install
!pip install spacy -U > /dev/null
!spacy download ru_core_news_sm > /dev/null

# Import
import re
import nltk
from nltk.tokenize import sent_tokenize, RegexpTokenizer
import spacy
import networkx as nx
from itertools import combinations
import pandas as pd

nlp = spacy.load("ru_core_news_sm")
nltk.download('punkt', quiet=True)

True

Выбираем текст для суммаризации:

In [50]:
text = """Старый кубинский рыбак Сантьяго 84 дня выходит в море и не может ничего поймать, поэтому его начинают считать salao[1], то есть самым что ни на есть невезучим. И только его маленький друг Манолин продолжает ему помогать, хотя отец запрещает ему рыбачить со старым Сантьяго и велит ходить в море с удачливыми рыбаками. Мальчик часто навещает старика в его хижине, помогает относить снасти, готовить еду, они часто разговаривают об американском бейсболе и их любимом игроке Джо Ди Маджио. Сантьяго говорит Манолину, что на следующий день он выйдет подальше в Гольфстрим, к северу от Кубы во Флоридский пролив, уверенный, что его полосе невезения должен наступить конец.
На 85-й день старик выходит в Гольфстрим, как обычно, на своей парусной лодке, забрасывает леску, и к полудню ему улыбается удача — на крючок попадается марлин около 5,5 метра длиной. Старик жалеет, что с ним нет мальчика: одному справиться нелегко. В течение двух дней и двух ночей марлин уносит лодку далеко в море. Мало поймать рыбу — с ней ещё надо доплыть до берега. Поранившись леской, Сантьяго понимает своего противника и сострадает ему, часто называя его братом. Он также утверждает, что никто не достоин съесть этого марлина из-за его благородства и достоинства.
На третий день рыба начинает плавать вокруг лодки. Изнурённый Сантьяго, практически в бреду, тратит свои последние силы, чтобы вытащить рыбу к поверхности и засадить в неё гарпун. Потом привязывает марлина к борту лодки и направляется домой, думая о высокой цене, которую он получит за неё на рынке, и о людях, которых он накормит.
На кровь из ран рыбы к лодке старика собираются акулы. Старик вступает с ними в схватку, убивает большую акулу-мако своим гарпуном, но теряет своё оружие. Он изготавливает новый гарпун, привязав свой нож к концу весла, чтобы отбиться от очередной атаки акул; таким способом он убивает пять акул, заставив остальных отступить. Но здесь силы неравны, и с наступлением ночи акулы пожирают почти всю тушу марлина, оставив от него лишь скелет из спинного хребта, хвоста и головы. Сантьяго понимает, что сейчас он стал совершенно невезучим, и, признавая поражение, говорит акулам, что они на самом деле убили человека и его мечты. Когда Сантьяго доплывает до берега перед рассветом следующего дня, он с трудом поднимается к своей хижине, взвалив тяжёлую мачту на плечо, а скелет рыбы оставив на берегу. Войдя в дом, он ложится на кровать и засыпает.
На следующий день вокруг лодки, к которой всё ещё был привязан рыбий скелет, собирается множество рыбаков. Один из рыбаков измеряет скелет верёвкой. Педрико забирает себе голову рыбы, а остальные рыбаки велят Манолину передать старику, что они сочувствуют ему. Туристы в соседнем кафе ошибочно принимают марлина за акулу. Манолин, переживая за старика, плачет, когда видит его израненные руки и убеждается, что тот дышит. Мальчик приносит в хижину газеты и кофе. Когда старик просыпается, они договариваются выйти в море ещё раз вместе. Заснув снова, Сантьяго видит во сне свою юность: львов на африканском побережье."""

Разбиваем текст на предложения:

In [51]:
sentences = sent_tokenize(text)
print(sentences[:3])

['Старый кубинский рыбак Сантьяго 84 дня выходит в море и не может ничего поймать, поэтому его начинают считать salao[1], то есть самым что ни на есть невезучим.', 'И только его маленький друг Манолин продолжает ему помогать, хотя отец запрещает ему рыбачить со старым Сантьяго и велит ходить в море с удачливыми рыбаками.', 'Мальчик часто навещает старика в его хижине, помогает относить снасти, готовить еду, они часто разговаривают об американском бейсболе и их любимом игроке Джо Ди Маджио.']


Из текста удаляем все последовательности, которые не подходят под регулярное выражение: \w т.е. [A-Za-z0-9_]

In [52]:

tokenizer = RegexpTokenizer(r'\w+')
words = [tokenizer.tokenize(sentence) for sentence in sentences]

sentences_lemma = []

for sentence in sentences:
    doc = nlp(sentence)
    lemmas = [token.lemma_ for token in doc if not token.is_stop]
    sentences_lemma.append(set(lemmas))

In [53]:
print(sentences_lemma[:2])

[{'считать', ']', ',', 'поймать', 'кубинский', 'самым', 'море', 'поэтому', '84', 'день', 'рыбак', 'ни', 'ничто', 'выходить', 'старый', 'невезучий', 'salao[1', '.', 'сантьяго', 'начинать'}, {'помогать', ',', 'удачливый', 'продолжать', 'маленький', 'ходить', 'море', 'велеть', 'хотя', 'запрещать', 'рыбачить', 'рыбак', 'отец', 'манолин', 'друг', 'старый', '.', 'сантьяго', 'со'}]


Определяем нашу меру сходства двух предложений:

In [54]:
def similarity(s1, s2):
    return len(s1.intersection(s2))/(1.0 * (len(s1) + len(s2)))

Для каждой возможной пары предложений в тексте вычисляется численное значение, измеряющее их схожесть

In [55]:
pairs = combinations(range(len(sentences)), 2)
scores = [(i, j, similarity(sentences_lemma[i], sentences_lemma[j])) for i, j in pairs]
# scores = filter(lambda x: x[2], scores)
print(scores[:2])

[(0, 1, 0.3076923076923077), (0, 2, 0.1)]


Создаем по данным значения граф:

In [57]:
g = nx.Graph()
g.add_weighted_edges_from(scores)

Применяем алгоритм PageRank:

In [58]:
pr = nx.pagerank(g)
result = sorted(((i, pr[i], s) for i, s in enumerate(sentences) if i in pr), 
                key=lambda x: pr[x[0]], reverse=True)

Сортируем наши предложения по индексу PageRank:

In [59]:
df = pd.DataFrame(result, columns=['Номер_предложения', 'Индекс_rank', 'Исходное_предложение']).sort_values(['Индекс_rank'], ascending=False)

In [60]:
df.head(5)

,Номер_предложения,Индекс_rank,Исходное_предложение
0,20,0.045055,"На следующий день вокруг лодки, к которой всё ..."
1,5,0.044173,"Старик жалеет, что с ним нет мальчика: одному ..."
2,26,0.042529,"Когда старик просыпается, они договариваются в..."
3,22,0.041079,"Педрико забирает себе голову рыбы, а остальные..."
4,13,0.039642,На кровь из ран рыбы к лодке старика собираютс...


Выбираем 4 самых значимых предложения и сортируем их в том порядке, в котором они встречаются в тексте. Это и будем "Кратким содержанием" исходного текста:

In [61]:
df[['Номер_предложения', 'Исходное_предложение']][:4].sort_values(['Номер_предложения'])['Исходное_предложение'].values

array(['Старик жалеет, что с ним нет мальчика: одному справиться нелегко.',
       'На следующий день вокруг лодки, к которой всё ещё был привязан рыбий скелет, собирается множество рыбаков.',
       'Педрико забирает себе голову рыбы, а остальные рыбаки велят Манолину передать старику, что они сочувствуют ему.',
       'Когда старик просыпается, они договариваются выйти в море ещё раз вместе.'],
      dtype=object)